In [2]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import time

#chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

# try:
#     driver = webdriver.Chrome(f'91/chromedriver')   
# except:
#     chromedriver_autoinstaller.install(True)
#     driver = webdriver.Chrome(f'91/chromedriver')

driver = webdriver.Chrome(f'../chromedriver')
driver.get("https://v4.map.naver.com")
driver.find_elements_by_css_selector("button.btn_close")[1].click()

driver.implicitly_wait(10)
#print(chrome_ver)

## store ids 
store_ids = [];


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=91.0.4472.101)


In [ ]:
store_id_path = 'store_id.txt'

def get_store_id(webl_obj) :
    for store in webl_obj:
        str_id = store.get_attribute("class") + ""
        store_ids.append(str_id.split(',')[1]);
        print(str_id.split(',')[1])
        WriteTxtFiles(store_id_path, str_id.split(',')[1])


def WriteTxtFiles(path, store_id):    
    with open(path, "a") as file:
        file.write(store_id + ",")
        file.close()



In [ ]:
search_query  = ['맛집']
search_box = driver.find_element_by_css_selector("input#search-input")
search_box.clear()
current_page_num = 1;
max_search_page_num = 10;

search_box.send_keys(search_query)
#         a:nth-child(3)#
search_button = driver.find_element_by_css_selector("button.spm")
search_button.click()
time.sleep(1)

# 우선 store_id는 100개만 수집
for i in range(max_search_page_num):
    next_button = driver.find_element_by_css_selector("div.paginate").find_elements_by_tag_name("a")    
    stores = driver.find_elements_by_css_selector("ul.lst_site>li")
    get_store_id(stores)
    current_page_num += 1;
    
    for nextItem in next_button:
        if(nextItem.text == "다음"):
            nextItem.click()
            time.sleep(1)
            break
            
        if(nextItem.text == str(current_page_num)):
            nextItem.click()
            time.sleep(1)
            break
 

In [5]:
# 수집된 store_id로 방문자 리뷰 데이터를 수집한다
from bs4 import BeautifulSoup
import requests

with open("store_id.txt", "r") as file:
    store_ids = file.read()
    file.close()
    
driver = webdriver.Chrome(f'../chromedriver')

for store_id in store_ids.split(","):
    if(store_id):
        print("================")
        url = f'https://pcmap.place.naver.com/restaurant/{store_id}/review/visitor'
        htm = driver.get(url)
        time.sleep(1)

        for i in range(10):
            # 더보기 버튼
            # 임시로 스크롤은 한 10번 정도만?            
            more_btn = driver.find_elements_by_class_name("_3iTUo")
            if(more_btn):
                more_btn[0].click()
                time.sleep(0.5)
            else:
                break;

        print("id : " + store_id)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # li _2Cv-r -> 부모 컨테이너 // span _2tObC  ->별점 // span WoYOw ->리뷰 내용

        for ll_item in soup.find_all('li',{'class' : '_2Cv-r'}):

            rating_ll = ll_item.find('span',{'class' : '_2tObC'})
            review_ll = ll_item.find('span',{'class' : 'WoYOw'})

            if(review_ll):
                #10자 이상 리뷰만 의미있는 값으로 본다.
                if(len(review_ll.text)>10):
                    with open("review_data.txt", "a") as file:
                        # id , 별점, 리뷰내용
                        # numpy.readTable 구조에 맞게 수정 필요.. 아예 CSV로 저장하던가..
                        file.write(store_id + rating_ll.text + review_ll.text + "\n") 
                    print("rating : " + rating_ll.text)
                    print("review : " + review_ll.text)
        file.close()
#        driver.close()
        

id : 1218049409
rating : 5
review : 사람이 항상 많지만 맛있어요
rating : 5
review : 엄청 바쁜데도 직원들이 전부 다 친절해요👍👍👍
rating : 3.5
review : Dyfigifhcjvjvjv
rating : 5
review : 샤로수길에서 내가 가장 좋아하는 가게!!
연인과 와도 좋고 친구랑 와도 좋은 곳~~~
데리고 왔던 사람들 다 만족하고 돌아감!
rating : 5
review : 맛있음 왜 사람 많은지 알 것 같음
rating : 4
review : 부타동  돼지고기   맛있네요. 불향과  적절한  단짠의  조합 좋아요.  다만  부타동  담은  그릇  이가  깨져있는데  작은것  하나까지  세심하게  고려해  주셨으면  하네요.
rating : 5
review : 오 플레이팅만 아기자기 이쁜 줄 알았더니 생각보다기대이상 맛있었어요 >_< 5시 땡치고 바로 갔는데 사람 많더라구요 샤로수길 가시면 한번 꼭 가보는걸 추천해요~!
rating : 5
review : 샤로수길에서 내가 가장 좋아하는 가게!!
연인과 와도 좋고 친구랑 와도 좋은 곳~~~
데리고 왔던 사람들 다 만족하고 돌아감!
rating : 5
review : 맛있음 왜 사람 많은지 알 것 같음
rating : 4
review : 부타동  돼지고기   맛있네요. 불향과  적절한  단짠의  조합 좋아요.  다만  부타동  담은  그릇  이가  깨져있는데  작은것  하나까지  세심하게  고려해  주셨으면  하네요.
rating : 5
review : 오 플레이팅만 아기자기 이쁜 줄 알았더니 생각보다기대이상 맛있었어요 >_< 5시 땡치고 바로 갔는데 사람 많더라구요 샤로수길 가시면 한번 꼭 가보는걸 추천해요~!
rating : 5
review : 생각보다 맛있어서 놀랐어요!
rating : 5
review : 부모님도 좋아하시네요~^^
rating : 4.5
review : 양은 좀 적지만 맛있었어요
rating : 3.5
review : 샤로수길

id : 1631471827
rating : 4
review : 한산해서 좋았어욥!!! 직원 친절세여
rating : 3
review : 분위기에 한번 와볼만 해요 가격대비 양은 작습니다 밀크티랑 동윤영 맛은 진하네요
rating : 3.5
review : 후르츠와플이 아주 맛난 곳 .. 근데 다른 테이블을 살펴보니 사람들마다 같은 음료인데도 잔이 다 다르더라고요 ..? 양도 다른건가..?ㅜ
rating : 4
review : 밀크티가 정말 맛있어요.
rating : 4
review : 과일 싱싱하고 와플이랑 잘 어울려요
커피가 좀 쓴맛이 있고.. 단점은 가격
rating : 5
review : 달콤 하면 서  고소함이  식감을  높여주네 요.
rating : 4
review : 후르츠는..비싸다고 생각하는데 동윤영이 맛있어요
rating : 5
review : 존맛 존맛 존맛 존맛
rating : 4
review : 후르츠는..비싸다고 생각하는데 동윤영이 맛있어요
rating : 5
review : 존맛 존맛 존맛 존맛
rating : 5
review : 동윤영은 여자친구가 골라서... 와플은 맛있을 예정입니다 어서 나왔으면 좋겠네요 ㅇㅅㅇ 
rating : 4
review : 후르츠와플 계절과일이라 딸기는 지금없고 체리랑 키위 포도 생크림 아주듬뿍~입니다 사진찍느라 시간이 지나서 그런지몰라도ㅋ 와플이 좀 눅눅했어요ㅠ 맛보단 비쥬얼인데 딸기가없어서 좀 아쉬웠어
요ㅋㅋ
rating : 5
review : 홍콩 길거리 음식 야시장에서 먹던 홍콩와플을 맛볼 수 있는 샤로수길 미드레벨입니다. 딸기, 청포도, 블루베리,생크림에 캐러멜라이징 한 바나나까지 와플후루츠 가격 12,000원! 달걀 모양 육각와플이 겉바속촉 고소하면서 맛있더군요.
rating : 5
review : 가격은 좀 사악하지만 분위기도 좋고 와플맛나용
rating : 4
review : 크림만 어마어마하게 올려놓고 로투스과자는 반쪼개서 꽂아놓았네여..
1인1음료라 많이들 들어 왔다가 나갑니다.

ra

id : 1290853152
rating : 4
review : 분위기도 나쁘지않고 좋았어요.
케이크들도 너무 귀렵고 전반적으로 많이 달달한 점 ~
rating : 5
review : 분위기도 좋고 얘기하기 좋아요ㅎㅎ
rating : 4.5
review : 디저트가 귀엽고 맛있어요!
rating : 4
review : 사람 정말 많아요 ㅋㅋ 웨이팅 길어서 포장해서 먹었어요 귀여운데 맛은 평범
rating : 4
review : sns 홍보대비 만족도는 아쉬워요ㅠㅠ (디저트 예뻐요, 가격대 있어요, 의자 불편해요ㅠㅠ)
rating : 3.5
review : 이쁘지만 맛은 그닥이였어요. 인스타용이네요. 
그래도 복숭아는 맛있었어요~ 
음료도 괜찮았습니다.
rating : 4
review : 인스타 보여주기용. 화장실 지저분해여..
rating : 4
review : 갬성값 개오지네요 ㅎㅎ
rating : 5
review : 예뻐요 근데 웨이팅이 엄청 길다는거 특히 3인은 더더
rating : 5
review : 곰돌이도 너무 귀엽고 음료도 완전 맛있어요! 강추!!
rating : 5
review : 의자는 좀 불편하지만 인테리어도 예쁘고 커피랑 디저트가 다 맛있어요
rating : 5
review : 여기서 2만8천원 썼네요 ㅋㅋ
곰돌이 별로예요
rating : 5
review : 청포도 타르트 정말 맛있어요 그런데 곰돌이는 별로네용 초콜렛 식감이 별로예요 뭉글뭉글..저렴한 초콜렛식감
rating : 1
review : 곰돌이 케이크 먹으러 왔는데 저녁 7시도 안됐는데 주 케이크 다 품절이네요....어쩔 수 없이 다른거 시켜 먹었어요ㅠㅠ 커피 맛은 밍밍하고 정말 별로였고.... 리얼초코 크로플은 먹을만했어요
rating : 4
review : 인스타 보여주기용. 화장실 지저분해여..
rating : 4
review : 갬성값 개오지네요 ㅎㅎ
rating : 5
review : 예뻐요 근데 웨이팅이 엄청 길다는거 특히 3인은 더더
rating : 5
review :

id : 1157452568
rating : 5
review : 늘 맛있어요~생각나는 맛~♡
rating : 5
review : 파스타가 진짜 맛있었어용
피자도 쫀득쫀득해서 안남기고 해치움ㅋㅋㅋ
rating : 5
review : 맛잇고 친절했어요!!ㅎㅎ
rating : 5
review : 맛있게 잘 먹었습니다 
rating : 5
review : 쿠폰 모아져서 윙 바꿔먹었음!!! 살짝매콤하고 짭짤해서 맛있네요.  피자는 언제나 맛있게 먹는 그 맛! 냠냠굿
rating : 5
review : 마가리타 넘 맛있어요~점보 사이즈는 사랑입니다ㅋㅋ맛있으면 0칼로리~
rating : 4
review : 피자는 동네 (흔하지만 꽤 괜찮은) 피잣집 수준이에요. 사이드로 시킨 스파게티도 평범한 미*터피자 오븐스파게티 정도? 하지만 나름 합리적인 가격대에, 다들 칭찬하듯 기본 도우가 치즈크러스트라는 
점이 치즈 좋아하는 사람으로서 너무 마음에 드네요👍👍 게다가 생각해보면 요즘 가볍게 들어가서 피자 먹고 (땡기면 간단히 피맥도 하고) 나올 만한 분위기의 가게가 많지 않은데, 밝고 가벼운 분위기
의 피잣집을 찾은 것 같아 좋았어요^^
rating : 5
review : 늘 친절하고 저렴한 가격에 맛까지 너무 좋습니다.
rating : 5
review : 자주 주문해 먹는데 피자가 항상 맛있어요 피자 가장자리에 적당한 식감의 치즈까지 ~ 뭐먹지? 고민할때 딱입니다
rating : 5
review : 너무 맛있고 사장님 친절하시고 가격도 괜찮고 재방문의사 100% 입니다~:)
rating : 5
review : 치즈와 토핑이 조화롭고 윙도 향이 좋고 간도 잘 맞습니다 맛있어요!
rating : 5
review : 고구마+허니 진짜 부드럽고 맛있어요
피자 도우도 안에 치즈가 부드럽게 들어있어서
맛있게 끝까지 ~
rating : 5
review : 늘 친절하고 저렴한 가격에 맛까지 너무 좋습니다.
rating : 5
review : 자주 주문해 먹는데 피자가 항상 맛있어요 피자 가장자리

id : 1862909287
rating : 0.5
review : 너무 맛있게 잘먹었습니다.  음식하시는분들도 정말 맛있고 빠르게 하시는거같아요~ 다만 토요일 점심타임 여자 알바생의 불친절한 응대로 가게는 다시 안갈것같네요;;  그냥 너무 많습니다.(음식 탁탁 
놓는거. 주문받는거. 수저 놓기. 웨이팅 대응. 대창 재료소진을 웨이팅 다한 후에 들어가기 직전에 알려줌. 음식 잘못 주문들어가서 잘못나왔는데 사과한마디없음 ) 등등 하나하나 기분나쁜 요소들이 너
무 많네요. 개선하면 정말 좋겠네요~^^ 알바생 이야기가 많던데 가게이미지를 위해서라도 꼭 개선해야할듯해요~~
rating : 3
review : 오랜단골이라 역시 맛있어요. 근데 홀담당이 너무 별로세요. 진심. 웨이팅하라고 문쾅하시는거부터 테이블도 대충 닦아주시고 물셋팅도 한참뒤에야 마지못해 해주시고. 전반적으로 매우 기분나쁜 식사
였어요.
rating : 1.5
review : 항상 줄이 길어서 궁금해서 방문했는데 특별한 맛은 없었고 무난했어요 가게 테이블이 너무 붙어있고 칸만이도 따로 없는데 코로나시국에 괜찮은지싶네요 무엇보다 다른 리뷰에도 나왔있던데 여자 알바
생인지 직원인지 모르겠지만 건성건성에 불친절해서 괜히 기분나쁘네요 직원응대에 신경써주세요
rating : 4
review : 맛집 리스트 보고 찾아갔어요
우동대창 맛있음
rating : 5
review : 맛있긴한데 맵찔이에겐 빡시다.. ㅜ 볶음밥 최고
rating : 3.5
review : 처음 숟가락 받을때 설거지가 안 되어있었어요 그래서 교체 받고 다시 받았는데 또 안되어 있어서 두번 교체 받았어요 음식은 육회가 너무 질겨요 밀면은 맛있었어요
rating : 4.5
review : 곰탕정식 맛나게 먹었습니다!
깔끔한 맛이네요^^
rating : 5
review : 서울에서 맛보는 부산~~~짱맛있어요!!!!!
rating : 5
review : 매운거 좋아하시는분들에게 추찬합니다 ㅎ 메장내 감성도 좋아요
rating : 5
review : 

...

id : 13342525
rating : 5
review : 로제 떡볶이도 맛있고 김밥도 맛있었어요! 
만두 역시 속이 꽉 차 잘 먹었습니다
rating : 3.5
review : 예전에 맛있어서 재방문 한건데..
떡볶이는.. 매웠고.. 4천원에 비해 양도 적었고.. 모듬튀김 보다는 따로 선택하는게 나을거 같아여 튀김옷도 두껍고..ㅠ 그나마 크런치롤만 먹을만 했어여ㅠ
rating : 4
review : 가격이 갑자기 다 500원씩 올랐어요ㅠㅠ
rating : 4
review : 배달앱으로 자주시켜먹어요
쌀떡볶기가  푸짐하고 맛있어요
rating : 0.5
review : 순대는 물에 담궜다가 뺀 듯 눅눅 물렁하고 떡볶이는 짜고 매운제육김밥은 바짝 쫄아있어서 제육이 들어간건가싶었다 먹다 버렸다
rating : 5
review : 맛있어요! 특히 쌀떡볶이 진짜 맛있어요!!
rating : 5
review : 돈까스도마싯구 전체적으로 좋음
rating : 5
review : 완전 존맛탱 꼭 가세요 두번 가세요 계속 가세요
rating : 5
review : 배달도 많이 하시는 거 같더라고욯ㅎ 남자분이 배달 준비하시던데 주문온 순서대로 줄지어 논거 보고 뭔가 괜히 기분이 좋아짐..ㅎㅎ
rating : 4.5
review : 매운라면은 신라면보다 더 매워욤
rating : 5
review : 배달도 빨리오고 맛있어요~~!
rating : 3
review : 조아용ㅎㅎ맛있게먹었어요
rating : 4
review : 순대가 제일 맛있어요
rating : 5
review : 맛있어요! 특히 쌀떡볶이 진짜 맛있어요!!
rating : 5
review : 돈까스도마싯구 전체적으로 좋음
rating : 5
review : 완전 존맛탱 꼭 가세요 두번 가세요 계속 가세요
rating : 5
review : 배달도 많이 하시는 거 같더라고욯ㅎ 남자분이 배달 준비하시던데 주문온 순서대로 줄지어 논거 보고 뭔가 괜히 기분이 좋아짐..ㅎㅎ
rating : 4.5
review : 매운라면

id : 37281466
rating : 5
review : 분위기에비해 안주와 와인가격이 합리적! 특히 파스타가 파스타 전문점보다 더 맛있었어요 💕ㅠㅠ 스테이크는 가격이 저렴해서 그런지 좀 아쉬웠어서 담엔 피자랑 파스타 시키려구요 ㅎㅎ
rating : 4
review : 와인가격도 합리적이고 장소도 역 근처라 가까워서 좋은것 같아요!
rating : 5
review : 음식다맛있어요~ 테라스자리 좋음
rating : 5
review : 분위기 완죤 좋아여~~
rating : 5
review : 와인 즐기기에 좋아요!
rating : 5
review : 분위기 좋고 조용해요~
근데 앞 판넬에 20%세일이라구 플레터 시켰는데 아닌가봐용 배안고파서 적당한 안주먹으려고 시켰더니 제가격내고 왔어요 ㅠ
rating : 4
review : 와인과 곁들인 음식 맛도 괜찮고 분위기도 좋아요
rating : 4.5
review : 와인도 맛있고 분위기도 좋아요
rating : 4.5
review : 간단히 와인마시기 좋아요
rating : 4
review : 우르메네타 맛있으니 꼭드세요 하몽 짜욧
rating : 4
review : 서울대입구역 근처 가본 와인집중 가장 괜찮음 와인 종류도 많고 가격대도 적당한 편
무엇보다 저렴한 와인도 어느정도 먹을만한 수준으로 가져다 놓아서 맘에듬
그치만 자리간격이 좁고 시끄러움
rating : 3.5
review : 로제파스타가 맛있어요. 넘 어둡고 좁아서 불편함
rating : 4.5
review : 늦은시간에 가서 북적거리지않게 잘먹고갑니당
rating : 4.5
review : 와인도 맛있고 분위기도 좋아요
rating : 4.5
review : 간단히 와인마시기 좋아요
rating : 4
review : 우르메네타 맛있으니 꼭드세요 하몽 짜욧
rating : 4
review : 서울대입구역 근처 가본 와인집중 가장 괜찮음 와인 종류도 많고 가격대도 적당한 편
무엇보다 저렴한 와인도 어느정도 먹을만한 수준으로 가져다 놓아서 맘에듬
그

id : 32358605
rating : 5
review : 연어 맛있네요 ㅎㅎㅎ
rating : 5
review : 연어 너무 사랑하는 나에게!!!! 연어가 왕창 먹고싶을때 찾는 곳 !!!!!! 여기 기름떡볶이 달달하고 조ㄴ맛!!!! 전에는 김치전 줬는데 이제는 감자튀김 주심,, 아무렴 오때용~ 존맛인디~~ 갈때마다 연어도 여러번 기떡도 여러번 추가하는곳 ,, 흐흐
rating : 2.5
review : 음식이 늦게 나오는편이고 맛은 쏘쏘라 리필안하고 나왔어요...기대보단 좀 아쉬웠어요ㅠ
rating : 5
review : 연어 너무 싱싱하고 맛있었고요
직원분들도 친절했어요
rating : 5
review : 관악구 에 이런곳 있다니...
 맛있게 잘 먹었어요~ㅎㅎ
연어 꿀맛~ㅎㅎ
rating : 5
review : 술 거하게 먹었네요ㅎㅎㅎㅎ
연어 육회 맛있네요 ~^^
rating : 4
review : 무한리필 연어라니 좋습니다🥰🥰
rating : 4
review : 연어 싱싱하고 맛있어요!
함께 나오는 떡볶이 옛날 떡꼬치 양념이라 맛나여
rating : 5
review : 연어 진짜 핵존맛
무한리필집 중에 최고!!
rating : 5
review : 술 거하게 먹었네요ㅎㅎㅎㅎ
연어 육회 맛있네요 ~^^
rating : 4
review : 무한리필 연어라니 좋습니다🥰🥰
rating : 4
review : 연어 싱싱하고 맛있어요!
함께 나오는 떡볶이 옛날 떡꼬치 양념이라 맛나여
rating : 5
review : 연어 진짜 핵존맛
무한리필집 중에 최고!!
rating : 5
review : 술 거하게 먹었네요ㅎㅎㅎㅎ
연어 육회 맛있네요 ~^^
rating : 4
review : 무한리필 연어라니 좋습니다🥰🥰
rating : 4
review : 연어 싱싱하고 맛있어요!
함께 나오는 떡볶이 옛날 떡꼬치 양념이라 맛나여
rating : 5
review : 연어 진짜 핵존맛
무한리필집 중에 최고!!
id : 1744964482
rating : 4
review : 지

id : 1104766439
rating : 4
review : 아메리카노 꽤 맛있네요 ㅋㅋ 디저트 전문점은 대체로 커피는 그저 그래서 기대 안했는데 생각보다 괜찮아서 놀람ㅋㅋ 아메리카노 산미 있어요 티라미슈는 걍 나쁘지않음
rating : 3.5
review : 쏘쏘 괜찮긴한데 엄청 유명할것 까지야 하는 느낌
rating : 2.5
review : 코로나 1시간 칼같이 지켜여 이건 좋은데 근데 망원홍차 보틀에 들어있는거 시켰는데 망티라떼가 나왔네요. 잘못들으셨다곤하는데... 남친이 원래 평소에도 뭐 시킬때 저한테 뭐먹을거라고 말 안해주는
편이고, 이 날도 주문할때 처음 뭐먹을지 듣고 아 홍차먹을거구나... 했었을 정도라서 잘못말했을리도 없는데 메뉴가 잘못 나온거 보고 좀 그랬네요 둘다 커피 안좋아하는편인데도 트러블 만들고 싶어하
지 않아서 남친이 억지로 그냥 마셨어요. 주문하실분들은 더블체크하시는게 좋을듯하네요 맛은 평범했습니다
rating : 3
review : 민트초코티라미수... 치즈가아니라 생크림으로 만든건가요 혹시? 민트초코생크림케이크 같습니다. 전 티라미수를 먹고 싶었던건데.....
rating : 0.5
review : 결론부터 말하자면, 문신하시고 단발머리에 파마하신 여성 알바생분. 심각하게 불친절해서 다신 가지 않으려고 합니다. 서울대입구점 오픈한지 얼마 안됐을 때부터, 알바 문의 할 정도로 너무 좋아했는
데요. 근래 너무 실망이 컸습니다. 티라미수 2개와 아아를 주문했더니 ‘커피는 포장만 가능하세요’라고 하시길래, ‘티라미수는 먹고 가도 되고, 커피는 포장만 되는건가요?’라고 여쭈었더니 짜증내며 ‘커
피는 포장만 가능하세요’라고 하시더라구요. 그래서 다시 여쭈었습니다. ‘.커피는 테이크 아웃 잔에만 담아주실 수 있다는 건가요?’ 그랬더니 ‘지금 구매하시는건 다 포장만 가능하세요’ 라며 짜증...
rating : 5
review : 티라미수, 딸기라떼 다 존맛! 근데 밀크티는 너무 달아요ㅠ
rating : 0.5
review : 웬만하면 영수

id : 1956805048
rating : 5
review : 친절하고 분위기와 와인리스트가 좋은 곳
rating : 1.5
review : 다시 가라고 하면 안감 분위기는 좋음 분위기만 좋음! 지저분하고 물 가져달라했는데도 잊고 주문도 누락하고 친절하지도 않음
rating : 4
review : 가성비 좋은 가격으로 즐길 수 있는 와인방이라고 생각합니다~~
rating : 4.5
review : 화이트 와인 맛있어요
rating : 4.5
review : 분위기도 좋고 음악도 좋은곳 :)
rating : 5
review : 조금 시끄럽지만 매장 분위기는 너무 좋아요
rating : 5
review : 분위기좋고 좌석이 별로 없어요!
근데 종업원을 불러도 잘 안옴 ㅠㅠ어둡거 해서 잘 안보이나봐요
rating : 4
review : 분위기 좋은 젊은 느낌 와인바
rating : 4
review : 와인을 얼음물에 담가 주는 점이 좋았고 메뉴판에 적힌 와인 설명이 친절해요 메뉴판 설명 꼼꼼히 보고 시켰네요.  안주가 싼 편입니다.
rating : 5
review : 와인이 저렴하고 치즈플레이트 좋아요
rating : 5
review : 샤로수길에서 제일 분위기 좋은 와인바! 가격도 착해요 :)
rating : 4
review : 분위기 좋은 와인바~ 메론맛나여~ 연인과 오기 좋은 곳 !
rating : 4.5
review : 글래스로 안팔아서 가격이 쫌 부담되지만...분위기는 신기해요
rating : 1
review : 와인 추천해달라고 했는데 직원들이 하나도 모르더라 와인 좋아하는 친구 데려갔다가 너무 민망했어요
rating : 5
review : 분위기 정말 끝장나요~!~! 정말 힙하고 인테리어가 너무 인상적이었어요! 어떤 자리는 영화를 볼 수 있게 되어있도 어떤자리는 침대를 리디자인해서 만들어놨는데 너무 좋아요! 와인과 곁들여 먹는 음식도 너무 맛있었어요! 서울대 와인바는 하라고지페 추천해요!
rating : 5
review : 인생 와인바 각 분위기도 넘